# 04- Structurer la sortie des LLMs
Dans cet exercice, nous allons nous concentrer sur la structuration des réponses des modèles de langage. Jusqu’ici, nous avons vu comment interagir avec un LLM, utiliser des prompts simples et introduire le raisonnement étape par étape avec les Chain of Thought. Cependant, les modèles renvoient souvent des réponses sous forme de texte libre, ce qui peut être difficile à exploiter de manière fiable dans une application.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [1]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')

## Configuration du llm sur dspy

In [2]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

En réalité, l’utilisation classique question -> answer n’est qu’un alias simplifié pour créer une interaction avec un LLM : derrière ce raccourci, il est possible de définir des signatures plus complètes basées sur des classes Python. Cela signifie que vous pouvez structurer vos prompts, vos entrées et vos sorties en utilisant des classes qui décrivent explicitement les types et les champs attendus, plutôt que de vous limiter à un texte libre. Cette approche rend vos appels plus robustes, plus lisibles et plus facilement exploitables par votre code, tout en conservant la flexibilité de générer des réponses riches depuis le modèle de langage.

<ins>**Exercice:**</ins> Modifier la signature et l'exécution pour extraire à partir du contenu de l'article `article_content` une liste de plat avec pour chacun le nom, la difficulté(Débutant, Intermédiaire ou Avancé), le temps de préparation et le temps de cuisson

##  Lecture de l'article

In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path("../../utils").resolve()))
from file_reader import read_file

article_content = read_file("../../assets/articles/002-article.md")

## Créer une signature

In [4]:
from enum import Enum
from pydantic import BaseModel, Field

class NiveauDifficulte(str, Enum):
    DEBUTANT = "Débutant"
    INTERMEDIAIRE = "Intermédiaire"
    AVANCE = "Avancé"

class Plat(BaseModel):
    nom: str = Field(description="Guide contenant différentes recettes de cuisine")
    difficulte: NiveauDifficulte = Field(description="Niveau de difficulté de réalisation du plat, si la valeur n'existe pas effectue le rapprochement avec une valeur")
    temps_preparation: int = Field(description="Temps de préparation à convertir en minutes")
    temps_cuisson: int = Field(description="Temps de cuisson à convertir en minutes")

class ExpertCulinaire(dspy.Signature):
    article_content: str = dspy.InputField(desc="Guide contenant différentes recettes de cuisine")
    plats: list[Plat] = dspy.OutputField(desc="Liste des plats extraits. IMPORTANT: le temps de préparation doit être converti en minutes")

## Exécuter la prédiction

In [5]:
from rich import print

assistant = dspy.Predict(ExpertCulinaire)
response = assistant(article_content=article_content)

print(response.plats)
dspy.inspect_history()
print(lm.history)

[
    Plat(
        nom='Ratatouille Provençale',
        difficulte=<NiveauDifficulte.INTERMEDIAIRE: 'Intermédiaire'>,
        temps_preparation=30,
        temps_cuisson=45
    ),
    Plat(
        nom='Pad Thaï Authentique',
        difficulte=<NiveauDifficulte.INTERMEDIAIRE: 'Intermédiaire'>,
        temps_preparation=25,
        temps_cuisson=10
    ),
    Plat(
        nom='Poulet Biryani',
        difficulte=<NiveauDifficulte.INTERMEDIAIRE: 'Intermédiaire'>,
        temps_preparation=40,
        temps_cuisson=50
    ),
    Plat(
        nom='Chili Con Carne Texas Style',
        difficulte=<NiveauDifficulte.DEBUTANT: 'Débutant'>,
        temps_preparation=15,
        temps_cuisson=150
    )
]





[2026-02-05T19:18:51.669291]

System message:

Your input fields are:
1. `article_content` (str): Guide contenant différentes recettes de cuisine
Your output fields are:
1. `plats` (list[Plat]): Liste des plats extraits. IMPORTANT: le temps de préparation doit être converti en minutes
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## article_content ## ]]
{article_content}

[[ ## plats ## ]]
{plats}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "$defs": {"NiveauDifficulte": {"type": "string", "enum": ["Débutant", "Intermédiaire", "Avancé"], "title": "NiveauDifficulte"}, "Plat": {"type": "object", "properties": {"difficulte": {"$ref": "#/$defs/NiveauDifficulte", "description": "Niveau de difficulté de réalisation du plat, si la valeur n'existe pas effectue le rapprochement avec une valeur"}, "nom": {"type": "string", "description": "Guide contenant différentes recettes de cuisine", "title"

[
    {
        'prompt': None,
        'messages': [
            {
                'role': 'system',
                'content': 'Your input fields are:\n1. `article_content` (str): Guide contenant différentes 
recettes de cuisine\nYour output fields are:\n1. `plats` (list[Plat]): Liste des plats extraits. IMPORTANT: le 
temps de préparation doit être converti en minutes\nAll interactions will be structured in the following way, with 
the appropriate values filled in.\n\n[[ ## article_content ## ]]\n{article_content}\n\n[[ ## plats ## ]]\n{plats}  
# note: the value you produce must adhere to the JSON schema: {"type": "array", "$defs": {"NiveauDifficulte": 
{"type": "string", "enum": ["Débutant", "Intermédiaire", "Avancé"], "title": "NiveauDifficulte"}, "Plat": {"type": 
"object", "properties": {"difficulte": {"$ref": "#/$defs/NiveauDifficulte", "description": "Niveau de difficulté de
réalisation du plat, si la valeur n\'existe pas effectue le rapprochement avec une valeur"}, "nom": {"type": 
"string", "description": "Guide contenant différentes recettes de cuisine", "title": "Nom"}, "temps_cuisson": 
{"type": "integer", "description": "Temps de cuisson à convertir en minutes", "title": "Temps Cuisson"}, 
"temps_preparation": {"type": "integer", "description": "Temps de préparation à convertir en minutes", "title": 
"Temps Preparation"}}, "required": ["nom", "difficulte", "temps_preparation", "temps_cuisson"], "title": "Plat"}}, 
"items": {"$ref": "#/$defs/Plat"}}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n  
Given the fields `article_content`, produce the fields `plats`.'
            },
            {
                'role': 'user',
                'content': "[[ ## article_content ## ]]\n# Guide Complet des Recettes de Cuisine Mondiale\n\n## 
Auteur\nMarie Laurent et Pierre Dubois  \nPublié le 15 mars 2023  \nDernière mise à jour : 10 janvier 
2024\n\n---\n\n## Résumé\nCe guide culinaire explore les recettes traditionnelles de différentes régions du monde. 
Chaque recette est accompagnée d'informations nutritionnelles, de variantes possibles, d'une liste d'ingrédients 
détaillée et d'instructions étape par étape. L'objectif est de permettre aux cuisiniers de tous niveaux de 
découvrir et reproduire des plats authentiques.\n\n---\n\n## Introduction\nLa cuisine est un art universel qui 
reflète l'histoire, la géographie et la culture des peuples. Ce guide rassemble 50 recettes provenant de 5 
continents, avec des niveaux de difficulté variés (débutant, intermédiaire, avancé) et des temps de préparation 
allant de 15 minutes à 4 heures.\n\n---\n\n## Recettes Européennes\n\n### Ratatouille Provençale\n**Origine** : 
France (Provence)  \n**Difficulté** : Intermédiaire  \n**Temps de préparation** : 30 min  \n**Temps de cuisson** : 
45 min  \n**Portions** : 6 personnes\n\n**Ingrédients** :\n- 2 aubergines moyennes (environ 600g)\n- 3 courgettes 
(environ 500g)\n- 4 tomates mûres (600g)\n- 2 poivrons rouges (300g)\n- 1 oignon jaune (150g)\n- 4 gousses d'ail\n-
100ml d'huile d'olive extra vierge\n- Herbes de Provence (thym, romarin, basilic)\n- Sel, 
poivre\n\n**Instructions** :\n1. Laver et couper tous les légumes en cubes de 2cm\n2. Faire revenir l'oignon et 
l'ail dans l'huile d'olive pendant 5 min\n3. Ajouter les aubergines et laisser cuire 10 min à feu moyen\n4. 
Incorporer les courgettes et poivrons, cuire 10 min supplémentaires\n5. Ajouter les tomates et les herbes, 
assaisonner\n6. Laisser mijoter 30 min à feu doux en remuant occasionnellement\n7. Servir chaud ou tiède avec du 
pain frais\n\n**Variantes** :\n- Version végétalienne : déjà végétalienne\n- Avec fromage : ajouter du fromage de 
chèvre émietté avant de servir\n- Version épicée : ajouter du piment d'Espelette\n\n**Informations 
nutritionnelles** (par portion) :\n- Calories : 180 kcal\n- Protéines : 4g\n- Glucides : 18g\n- Lipides : 12g\n- 
Fibres : 6g\n\n\n---\n\n\n## Recettes Asiatiques\n\n### Pad Thaï Authentique\n**Origine** : T